## 1 - Importação de Bibliotecas

In [0]:
import pyspark.sql.functions as f

### 1.1 - Função que nomeia os arquivos

In [0]:
def move_files(ext, source, target, new_name):
  files = dbutils.fs.ls(source)
  for i in range (0, len(files)):
    file = files[i].name
    if ext in file:
      dbutils.fs.cp(files[i].path, target + new_name + '.' + ext)
      print ('Nome Origem: ' + file + '\nNome Destino: ' +  new_name + '.' + ext)

#### 1.1.1 - Variáveis de versionamento

In [0]:
path_dev_bi_dia = "/mnt/data/serving/fato_vol_margem_bi/fato_vol_margem_bi_dia/"
path_dev_bi_mes = "/mnt/data/serving/fato_vol_margem_bi/fato_vol_margem_bi_mes/"
path_target = "/mnt/data/serving/fato_vol_margem_bi/"

## 2 - Criação do DataFrame

In [0]:
df_dm_componente = spark.read.parquet('/mnt/data/raw/dados_internos/bi/clientes/dimensao/rw_dm_componente.parquet')

In [0]:
dm_produto_ge = spark.read.parquet('/mnt/data/serving/sv_dm_produto_ge.parquet', infer_schema=True)

In [0]:
dm_tipo_negocio = spark.read.parquet('/mnt/data/raw/dados_internos/bi/clientes/dimensao/rw_dm_tipo_negocio.parquet')

In [0]:
df_fato_vol_margem_bi = spark.read.parquet('/mnt/data/refined/tabela_unificada_ds/ag_ind_vnd_comp_prod_dia_ds/*/*/*/rf_ag_ind_vnd_comp_prod_dia_ds_***.parquet')

## 3 - Tratamento dos Dados

In [0]:
dm_produto_ge = dm_produto_ge.filter(f.col("CD_GRP_PROD_VND_NIV_3").isin("2", "3", "4"))

In [0]:
df_fato_vol_margem_bi_dia = df_fato_vol_margem_bi.filter((f.col('DT_REF') >= '2021-01-01') & (f.col('NO_UNIDNEG') == 2))

In [0]:
df_fato_vol_margem_bi_mes = df_fato_vol_margem_bi.filter((f.col('DT_REF') >= '2018-01-01') & (f.col('DT_REF') < '2021-01-01') & (f.col('NO_UNIDNEG') == 2))

In [0]:
dm_tipo_negocio = dm_tipo_negocio.selectExpr('cast(CD_TPNEG_ANT as int)', 'cast(CD_TPNEG as int)', 'cast(CD_GRP_TPNEG as int)', 'DS_TPNEG')

In [0]:
df_fato_vol_margem_bi_mes = (
                              df_fato_vol_margem_bi_mes
                               .withColumn("NO_AM", f.date_format(f.col('DT_REF'),"yyyyMM"))
                            )

In [0]:
df_dm_componente = (
                     df_dm_componente
                      .withColumnRenamed('CD_TPNEG', 'CD_TPNEG_DM_COMP')
                   )

In [0]:
df_dm_componente = df_dm_componente.selectExpr('cast(CD_TPNEG_DM_COMP as int)', 'cast(CD_COMP as int)', 'cast(CD_PESS as int)', 'cast(CD_PTOV as int)')

In [0]:
df_fato_vol_margem_bi_mes = (
                              df_fato_vol_margem_bi_mes
                               .join(df_dm_componente, (df_dm_componente.CD_COMP == df_fato_vol_margem_bi_mes.CD_COMP), how='left')
                               .join(dm_tipo_negocio, (df_dm_componente.CD_TPNEG_DM_COMP == dm_tipo_negocio.CD_TPNEG), how='left')
                               .join(dm_produto_ge, (dm_produto_ge.CD_PROD == df_fato_vol_margem_bi_mes.CD_PROD), how='inner')
                               .drop(df_dm_componente.CD_PESS)
                               .drop(dm_produto_ge.CD_PROD)
                               .withColumn('TIPO_NEGOCIO', f.when(f.col('CD_GRP_TPNEG').isin(6,10), 'Rede')
                                                            .otherwise('BB'))
                               .select('CD_PESS', 'CD_PTOV', 'NO_AM', 'CD_PROD', 'CD_UNID_ORG', 'TIPO_NEGOCIO', 'VO_PROD_AMB', 'VR_MRGT', 'VR_TOT_AVCRED', 
                                       'VR_TOT_BONIF_POS')
                               .groupBy('NO_AM', 'CD_PTOV', 'CD_PESS', 'CD_PROD', 'CD_UNID_ORG', 'TIPO_NEGOCIO')
                               .sum('VO_PROD_AMB', 'VR_MRGT', 'VR_TOT_AVCRED', 'VR_TOT_BONIF_POS')
                            )

In [0]:
df_fato_vol_margem_bi_mes = (
                              df_fato_vol_margem_bi_mes
                               .withColumnRenamed('sum(VO_PROD_AMB)', 'VOL')
                               .withColumnRenamed('sum(VR_MRGT)', 'MG_COMERCIAL')
                               .withColumnRenamed('sum(VR_TOT_AVCRED)', 'MG_AVIC')
                               .withColumnRenamed('sum(VR_TOT_BONIF_POS)', 'MG_CPOSTBONIF')  
                             )

In [0]:
df_fato_vol_margem_bi_mes = df_fato_vol_margem_bi_mes.selectExpr('CD_PESS', 'CD_PTOV', 'NO_AM', 'CD_PROD', 'CD_UNID_ORG', 'TIPO_NEGOCIO', 'round(VOL, 4)', 'round(MG_COMERCIAL, 4)', 'round(MG_AVIC, 4)', 'round(MG_CPOSTBONIF, 4)')

In [0]:
df_fato_vol_margem_bi_mes = (
                              df_fato_vol_margem_bi_mes
                               .withColumnRenamed('round(VOL, 4)', 'VOL')
                               .withColumnRenamed('round(MG_COMERCIAL, 4)', 'MG_COMERCIAL')
                               .withColumnRenamed('round(MG_AVIC, 4)', 'MG_AVIC')
                               .withColumnRenamed('round(MG_CPOSTBONIF, 4)', 'MG_CPOSTBONIF')  
                             )

In [0]:
df_fato_vol_margem_bi_dia = (
                              df_fato_vol_margem_bi_dia
                               .join(df_dm_componente, (df_dm_componente.CD_COMP == df_fato_vol_margem_bi_dia.CD_COMP), how='left')
                               .join(dm_tipo_negocio, (df_dm_componente.CD_TPNEG_DM_COMP == dm_tipo_negocio.CD_TPNEG), how='left')
                               .join(dm_produto_ge, (dm_produto_ge.CD_PROD == df_fato_vol_margem_bi_dia.CD_PROD), how='inner')
                               .drop(df_dm_componente.CD_PESS)
                               .drop(dm_produto_ge.CD_PROD)
                               .withColumn('TIPO_NEGOCIO', f.when(f.col('CD_GRP_TPNEG').isin(6,10), 'Rede')
                                                            .otherwise('BB'))
                               .select('CD_PESS', 'CD_PTOV', 'DT_REF', 'CD_PROD', 'CD_UNID_ORG', 'TIPO_NEGOCIO', 'VO_PROD_AMB', 'VR_MRGT', 'VR_TOT_AVCRED', 
                                       'VR_TOT_BONIF_POS')
                               .groupBy('DT_REF', 'CD_PTOV', 'CD_PESS', 'CD_PROD', 'CD_UNID_ORG', 'TIPO_NEGOCIO')
                               .sum('VO_PROD_AMB', 'VR_MRGT', 'VR_TOT_AVCRED', 'VR_TOT_BONIF_POS')
                            )

In [0]:
df_fato_vol_margem_bi_dia = (
                              df_fato_vol_margem_bi_dia
                               .withColumnRenamed('sum(VO_PROD_AMB)', 'VOL')
                               .withColumnRenamed('sum(VR_MRGT)', 'MG_COMERCIAL')
                               .withColumnRenamed('sum(VR_TOT_AVCRED)', 'MG_AVIC')
                               .withColumnRenamed('sum(VR_TOT_BONIF_POS)', 'MG_CPOSTBONIF')  
                             )

In [0]:
df_fato_vol_margem_bi_dia = df_fato_vol_margem_bi_dia.selectExpr('CD_PESS', 'CD_PTOV', 'DT_REF', 'CD_PROD', 'CD_UNID_ORG', 'TIPO_NEGOCIO', 'round(VOL, 4)', 'round(MG_COMERCIAL, 4)', 'round(MG_AVIC, 4)', 'round(MG_CPOSTBONIF, 4)')

In [0]:
df_fato_vol_margem_bi_dia = (
                              df_fato_vol_margem_bi_dia
                               .withColumnRenamed('round(VOL, 4)', 'VOL')
                               .withColumnRenamed('round(MG_COMERCIAL, 4)', 'MG_COMERCIAL')
                               .withColumnRenamed('round(MG_AVIC, 4)', 'MG_AVIC')
                               .withColumnRenamed('round(MG_CPOSTBONIF, 4)', 'MG_CPOSTBONIF')  
                             )

## 4 - Output

In [0]:
df_fato_vol_margem_bi_mes\
.coalesce(1)\
.write\
.format("parquet")\
.mode("overwrite")\
.save(path_dev_bi_mes, header=True)

In [0]:
ext="parquet"
source_bi_mes=path_dev_bi_mes
target=path_target
new_name="sv_fato_vol_margem_bi_mes"

move_files(ext, source_bi_mes, target, new_name)

Nome Origem: part-00000-tid-7299421301109024908-9534832b-ae02-499e-b3c3-5e3975ebadc0-63881-1-c000.snappy.parquet
Nome Destino: sv_fato_vol_margem_bi_mes.parquet

In [0]:
df_fato_vol_margem_bi_dia\
.coalesce(1)\
.write\
.format("parquet")\
.mode("overwrite")\
.save(path_dev_bi_dia, header=True)

In [0]:
ext="parquet"
source_bi_dia=path_dev_bi_dia
target=path_target
new_name="sv_fato_vol_margem_bi_dia"

move_files(ext, source_bi_dia, target, new_name)

Nome Origem: part-00000-tid-8790875728698162630-f9ce6a1e-720b-496a-84e8-15a7a5c709cd-63954-1-c000.snappy.parquet
Nome Destino: sv_fato_vol_margem_bi_dia.parquet